In [1]:
import json
import re
import csv
import time
import random
import glob
from lxml import etree
import requests
from selenium import webdriver
from time import sleep

In [2]:
print(random.random()*30)

2.0651534388521133


In [3]:
data={
    'SIGIR':['https://dl.acm.org/doi/proceedings/10.1145/1835449',
             'https://dl.acm.org/doi/proceedings/10.1145/2009916',
             'https://dl.acm.org/doi/proceedings/10.1145/2348283',
             'https://dl.acm.org/doi/proceedings/10.1145/2484028',
             'https://dl.acm.org/doi/proceedings/10.1145/2600428',
             'https://dl.acm.org/doi/proceedings/10.1145/2766462',
             'https://dl.acm.org/doi/proceedings/10.1145/2911451',
             'https://dl.acm.org/doi/proceedings/10.1145/3077136',
             'https://dl.acm.org/doi/proceedings/10.1145/3209978',
             'https://dl.acm.org/doi/proceedings/10.1145/3331184'],
    'SIGKDD':['https://dl.acm.org/doi/proceedings/10.1145/1835804',
              'https://dl.acm.org/doi/proceedings/10.1145/2020408',
              'https://dl.acm.org/doi/proceedings/10.1145/2339530',
              'https://dl.acm.org/doi/proceedings/10.1145/2487575',
              'https://dl.acm.org/doi/proceedings/10.1145/2623330',
              'https://dl.acm.org/doi/proceedings/10.1145/2783258',
              'https://dl.acm.org/doi/proceedings/10.1145/2939672',
              'https://dl.acm.org/doi/proceedings/10.1145/3097983',
              'https://dl.acm.org/doi/proceedings/10.1145/3219819',
              'https://dl.acm.org/doi/proceedings/10.1145/3292500'],
    'CIKM':['https://dl.acm.org/doi/proceedings/10.1145/1871437',
            'https://dl.acm.org/doi/proceedings/10.1145/2063576',
            'https://dl.acm.org/doi/proceedings/10.1145/2396761',
            'https://dl.acm.org/doi/proceedings/10.1145/2505515',
            'https://dl.acm.org/doi/proceedings/10.1145/2661829',
            'https://dl.acm.org/doi/proceedings/10.1145/2806416',
            'https://dl.acm.org/doi/proceedings/10.1145/2983323',
            'https://dl.acm.org/doi/proceedings/10.1145/3132847',
            'https://dl.acm.org/doi/proceedings/10.1145/3269206',
            'https://dl.acm.org/doi/proceedings/10.1145/3357384'],
    'WWW':['https://dl.acm.org/doi/proceedings/10.1145/1772690',
           'https://dl.acm.org/doi/proceedings/10.1145/1963405',
           'https://dl.acm.org/doi/proceedings/10.1145/2187836',
           'https://dl.acm.org/doi/proceedings/10.1145/2488388',
           'https://dl.acm.org/doi/proceedings/10.1145/2566486',
           'https://dl.acm.org/doi/proceedings/10.5555/2736277',
           'https://dl.acm.org/doi/proceedings/10.5555/2872427',
           'https://dl.acm.org/doi/proceedings/10.5555/3038912',
           'https://dl.acm.org/doi/proceedings/10.5555/3178876',
           'https://dl.acm.org/doi/proceedings/10.1145/3308558'],
    'WSDM':['https://dl.acm.org/doi/proceedings/10.1145/1718487',
            'https://dl.acm.org/doi/proceedings/10.1145/1935826',
            'https://dl.acm.org/doi/proceedings/10.1145/2124295',
            'https://dl.acm.org/doi/proceedings/10.1145/2433396',
            'https://dl.acm.org/doi/proceedings/10.1145/2556195',
            'https://dl.acm.org/doi/proceedings/10.1145/2684822',
            'https://dl.acm.org/doi/proceedings/10.1145/2835776',
            'https://dl.acm.org/doi/proceedings/10.1145/3018661',
            'https://dl.acm.org/doi/proceedings/10.1145/3159652',
            'https://dl.acm.org/doi/proceedings/10.1145/3289600']
}
filename='target_url.json'
with open(filename,'w') as f:
    json.dump(data,f)
with open(filename) as f:
    data2=json.load(f)

In [4]:
confs = ['SIGIR','SIGKDD','WWW','CIKM','WSDM']

In [5]:
user_agent = [
    "Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_6_8; en-us) AppleWebKit/534.50 (KHTML, like Gecko) Version/5.1 Safari/534.50",
    "Mozilla/5.0 (Windows; U; Windows NT 6.1; en-us) AppleWebKit/534.50 (KHTML, like Gecko) Version/5.1 Safari/534.50",
    "Mozilla/5.0 (Windows NT 10.0; WOW64; rv:38.0) Gecko/20100101 Firefox/38.0",
    "Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; .NET4.0C; .NET4.0E; .NET CLR 2.0.50727; .NET CLR 3.0.30729; .NET CLR 3.5.30729; InfoPath.3; rv:11.0) like Gecko",
    "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Trident/5.0)",
    "Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0)",
    "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0)",
    "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1)",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.6; rv:2.0.1) Gecko/20100101 Firefox/4.0.1",
    "Mozilla/5.0 (Windows NT 6.1; rv:2.0.1) Gecko/20100101 Firefox/4.0.1",
    "Opera/9.80 (Macintosh; Intel Mac OS X 10.6.8; U; en) Presto/2.8.131 Version/11.11",
    "Opera/9.80 (Windows NT 6.1; U; en) Presto/2.8.131 Version/11.11",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_0) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.56 Safari/535.11",
    "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; Maxthon 2.0)",
    "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; TencentTraveler 4.0)",
    "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)",
    "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; The World)",
    "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; Trident/4.0; SE 2.X MetaSr 1.0; SE 2.X MetaSr 1.0; .NET CLR 2.0.50727; SE 2.X MetaSr 1.0)",
    "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; 360SE)",
    "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; Avant Browser)",
    "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)",
    "Mozilla/5.0 (iPhone; U; CPU iPhone OS 4_3_3 like Mac OS X; en-us) AppleWebKit/533.17.9 (KHTML, like Gecko) Version/5.0.2 Mobile/8J2 Safari/6533.18.5",
    "Mozilla/5.0 (iPod; U; CPU iPhone OS 4_3_3 like Mac OS X; en-us) AppleWebKit/533.17.9 (KHTML, like Gecko) Version/5.0.2 Mobile/8J2 Safari/6533.18.5",
    "Mozilla/5.0 (iPad; U; CPU OS 4_3_3 like Mac OS X; en-us) AppleWebKit/533.17.9 (KHTML, like Gecko) Version/5.0.2 Mobile/8J2 Safari/6533.18.5",
    "Mozilla/5.0 (Linux; U; Android 2.3.7; en-us; Nexus One Build/FRF91) AppleWebKit/533.1 (KHTML, like Gecko) Version/4.0 Mobile Safari/533.1",
    "MQQBrowser/26 Mozilla/5.0 (Linux; U; Android 2.3.7; zh-cn; MB200 Build/GRJ22; CyanogenMod-7) AppleWebKit/533.1 (KHTML, like Gecko) Version/4.0 Mobile Safari/533.1",
    "Opera/9.80 (Android 2.3.4; Linux; Opera Mobi/build-1107180945; U; en-GB) Presto/2.8.149 Version/11.10",
    "Mozilla/5.0 (Linux; U; Android 3.0; en-us; Xoom Build/HRI39) AppleWebKit/534.13 (KHTML, like Gecko) Version/4.0 Safari/534.13",
    "Mozilla/5.0 (BlackBerry; U; BlackBerry 9800; en) AppleWebKit/534.1+ (KHTML, like Gecko) Version/6.0.0.337 Mobile Safari/534.1+",
    "Mozilla/5.0 (hp-tablet; Linux; hpwOS/3.0.0; U; en-US) AppleWebKit/534.6 (KHTML, like Gecko) wOSBrowser/233.70 Safari/534.6 TouchPad/1.0",
    "Mozilla/5.0 (SymbianOS/9.4; Series60/5.0 NokiaN97-1/20.0.019; Profile/MIDP-2.1 Configuration/CLDC-1.1) AppleWebKit/525 (KHTML, like Gecko) BrowserNG/7.1.18124",
    "Mozilla/5.0 (compatible; MSIE 9.0; Windows Phone OS 7.5; Trident/5.0; IEMobile/9.0; HTC; Titan)",
    "UCWEB7.0.2.37/28/999",
    "NOKIA5700/ UCWEB7.0.2.37/28/999",
    "Openwave/ UCWEB7.0.2.37/28/999",
    "Mozilla/4.0 (compatible; MSIE 6.0; ) Opera/UCWEB7.0.2.37/28/999",
    "Mozilla/6.0 (iPhone; CPU iPhone OS 8_0 like Mac OS X) AppleWebKit/536.26 (KHTML, like Gecko) Version/8.0 Mobile/10A5376e Safari/8536.25"]


def write_file(container,conf,tag):
    file_path = r"C:\Users\haiyuanzhao\Desktop\result\{}.csv".format(conf+'_'+str(tag))
    #print(file_path)
    with open(file_path,'w',newline="") as f:
        writer=csv.writer(f)
        for i in container:
            temp=[]
            temp.append(i)
            writer.writerow(temp)

def crawl_proceeding(proceeding_url,conf,tag):
    start = time.time()
    paper_url_list =[]
    header1 = {'User-Agent': random.choice(user_agent)}
    proceeding_page = requests.get(proceeding_url,headers=header1)
    proceeding_html = etree.HTML(proceeding_page.text)
    issue_url_list = proceeding_html.xpath("//div[@class='toc__section accordion-tabbed__tab']//a[@class='section__title accordion-tabbed__control left-bordered-title']/@href")
    sleep(random.random()*30)
    for issue_url in issue_url_list:
        header2 = {'User-Agent': random.choice(user_agent)}
        issue_page = requests.get("https://dl.acm.org"+issue_url,headers=header2)
        issue_html = etree.HTML(issue_page.text)
        paper_url = issue_html.xpath("//h5[@class='issue-item__title']/a/@href")
        paper_url_list.extend(paper_url)
        print(issue_url)
        #print(paper_url)
        sleep(random.random()*30)
    write_file(paper_url_list,conf,tag)
    end = time.time()
    cost_time = end-start
    print(conf,'proceeding',tag,' time:',cost_time)
    return cost_time

def crawl_proceeding2(proceeding_url,conf,tag):
    start = time.time()
    paper_url_list =[]
    pid=1
    for delta in range(13):
        header1 = {'User-Agent': random.choice(user_agent)}
        n_pid = pid+30*(delta)
        proceeding_page = requests.get(proceeding_url+'?id='+str(n_pid),headers=header1)
        proceeding_html = etree.HTML(proceeding_page.text)
        #issue_url_list = proceeding_html.xpath("//div[@class='toc__section accordion-tabbed__tab']//a[@class='section__title accordion-tabbed__control left-bordered-title']/@href")
        paper_url = proceeding_html.xpath("//h5[@class='issue-item__title']/a/@href")
        paper_url_list.extend(paper_url)
        print(proceeding_url+'?id='+str(n_pid))
        sleep(random.random()*30)
    write_file(paper_url_list,conf,tag)
    end = time.time()
    cost_time = end-start
    print(conf,'proceeding',tag,' time:',cost_time)
    return cost_time

In [10]:
for conf in ['WSDM']:
    tag =2018
    for proceeding_url in data2[conf][8:10]:
        crawl_proceeding(proceeding_url,conf,tag)
        sleep(random.random()*600)
        tag= tag + 1

/doi/proceedings/10.1145/3159652?tocHeading=heading1
/doi/proceedings/10.1145/3159652?tocHeading=heading2
/doi/proceedings/10.1145/3159652?tocHeading=heading3
/doi/proceedings/10.1145/3159652?tocHeading=heading4
/doi/proceedings/10.1145/3159652?tocHeading=heading5
/doi/proceedings/10.1145/3159652?tocHeading=heading6
/doi/proceedings/10.1145/3159652?tocHeading=heading7
WSDM proceeding 2018  time: 166.7505395412445
/doi/proceedings/10.1145/3289600?tocHeading=heading1
/doi/proceedings/10.1145/3289600?tocHeading=heading2
/doi/proceedings/10.1145/3289600?tocHeading=heading3
/doi/proceedings/10.1145/3289600?tocHeading=heading4
/doi/proceedings/10.1145/3289600?tocHeading=heading5
/doi/proceedings/10.1145/3289600?tocHeading=heading6
/doi/proceedings/10.1145/3289600?tocHeading=heading7
/doi/proceedings/10.1145/3289600?tocHeading=heading8
/doi/proceedings/10.1145/3289600?tocHeading=heading9
/doi/proceedings/10.1145/3289600?tocHeading=heading10
/doi/proceedings/10.1145/3289600?tocHeading=heading1

In [23]:
crawl_proceeding2(data2['WWW'][9],'WWW',2019)

https://dl.acm.org/doi/proceedings/10.1145/3308558?id=1
https://dl.acm.org/doi/proceedings/10.1145/3308558?id=31
https://dl.acm.org/doi/proceedings/10.1145/3308558?id=61
https://dl.acm.org/doi/proceedings/10.1145/3308558?id=91
https://dl.acm.org/doi/proceedings/10.1145/3308558?id=121
https://dl.acm.org/doi/proceedings/10.1145/3308558?id=151
https://dl.acm.org/doi/proceedings/10.1145/3308558?id=181
https://dl.acm.org/doi/proceedings/10.1145/3308558?id=211
https://dl.acm.org/doi/proceedings/10.1145/3308558?id=241
https://dl.acm.org/doi/proceedings/10.1145/3308558?id=271
https://dl.acm.org/doi/proceedings/10.1145/3308558?id=301
https://dl.acm.org/doi/proceedings/10.1145/3308558?id=331
https://dl.acm.org/doi/proceedings/10.1145/3308558?id=361
WWW proceeding 2019  time: 264.87295150756836


264.87295150756836

In [6]:
FILEPATH = r'C:\Users\haiyuanzhao\Desktop\result\\'
FILETYPE = r'.csv'
file_list=glob.glob(FILEPATH+'*'+FILETYPE)


def data_loader(filepath):
    with open(filepath) as f:
        reader = csv.reader(f)
        paper_list = [r for r in reader]
    return paper_list

def clean_url(paper_list):
    clean_list =[]
    for url in paper_list:
        try:
            match_item = re.search( '/doi/abs/\d\d[.]\d+/\d+[.]\d+',url[0]).group()
            clean_list.append(match_item)
        except:
            continue
    return clean_list

def clean_ref_url(paper_ref_urls):
    clean_list =[]
    for url in paper_ref_urls:
        try:
            match_item = re.search( '\d\d[.]\d+/\d+[.]\d+',url).group()
            clean_list.append(match_item)
        except:
            continue
    return clean_list

def jud_in(ref_paper,all_paper_set):
    jud_result = []
    for doi in ref_paper:
        jud_result.append((doi in all_paper_set))
    return jud_result

def write_file2(container,conf,yr):
    file_path = r"C:\Users\haiyuanzhao\Desktop\result\{}.json".format(conf+'_'+str(yr))
    with open(file_path,'w') as f:
        json.dump(container,f)

def paper_crawler(conf,yr,paper_url,all_paper):
    all_paper_set = set(all_paper)
    print('connecting...')
    header = {'User-Agent': random.choice(user_agent)}
    paper_page = requests.get(paper_url,headers=header)
    print('success...')
    paper_html = etree.HTML(paper_page.text)
    
    paper_doi = re.search( '\d\d[.]\d+/\d+[.]\d+',paper_url).group()
    paper_conf = conf
    paper_yr = yr
    paper_title = paper_html.xpath("//h1[@class='citation__title']/text()")
    paper_abstract = paper_html.xpath("//div[@class='article__section article__abstract hlFld-Abstract']/p/text()")
    
    paper_ref_urls = paper_html.xpath("//span[@class='references__suffix']/a/@href")
    cleaned_ref_paper = clean_ref_url(paper_ref_urls)
    jud_result = jud_in(cleaned_ref_paper,all_paper_set)
    paper_ref_list = []
    for i in range(len(cleaned_ref_paper)):
        dict_cell={}
        dict_cell['ref_doi']=cleaned_ref_paper[i]
        dict_cell['ref_in']=jud_result[i]
        paper_ref_list.append(dict_cell)
        
    author_doi = paper_html.xpath("//li[@class='loa__item']/div/div[@class='author-info__body']/p/@data-doi")
    author_org = paper_html.xpath("//li[@class='loa__item']/div/div[@class='author-info__body']/p/text()")
    author_info_list = []
    for i in range(len(author_doi)):
        dict_cell={}
        dict_cell['author_doi']=author_doi[i]
        dict_cell['author_org']=author_org[i]
        author_info_list.append(dict_cell)
    #print(len(author_info_list))
    author_info_list_n=[]
    for i in range(int(len(author_info_list)/2)):
        author_info_list_n.append(author_info_list[i])
    
    paper_info={}
    paper_info['paper_doi']=paper_doi
    paper_info['paper_url']=paper_url
    paper_info['paper_conf']=paper_conf
    paper_info['paper_yr']=paper_yr
    paper_info['paper_title']=paper_title
    paper_info['paper_abstract']=paper_abstract
    paper_info['paper_ref_list']=paper_ref_list
    paper_info['author_info_list']=author_info_list_n
    
    return paper_info
    
all_paper=[]
for fp in file_list:
    paper_list = data_loader(fp)
    clean_list = clean_url(paper_list)
    paper_doi = [re.search( '\d\d[.]\d+/\d+[.]\d+',url).group() for url in clean_list]
    all_paper.extend(paper_doi)

In [ ]:
paper_crawler('CIKM',2017,'https://dl.acm.org/doi/abs/10.1145/3132847.3133027',all_paper)

In [7]:
FILEPATH = r'C:\Users\haiyuanzhao\Desktop\result\\'
FILETYPE = r'.csv'
for conf in ['WSDM']:
    for yr in range(2010,2020):
        filepath = FILEPATH+conf+'_'+str(yr)+FILETYPE
        paper_list = data_loader(filepath)
        clean_list = clean_url(paper_list)
        st = time.time()
        paper_info_list=[]
        for paper_url in clean_list:
            full_url = 'https://dl.acm.org'+paper_url
            #print(full_url)
            try:
                paper_info = paper_crawler(conf,yr,full_url,all_paper)
                paper_info_list.append(paper_info)
                if not len(paper_info_list)%10:
                    write_file2(paper_info_list,conf,yr)
                sleep(random.random()*30)
                print(full_url)
            except:
                print('Time Out, Sleep 5 min')
                write_file2(paper_info_list,conf,yr)
                sleep(300)
                continue
        write_file2(paper_info_list,conf,yr)
        ed = time.time()
        print(conf,yr,ed-st)

connecting...
success...
https://dl.acm.org/doi/abs/10.1145/1718487.1718489
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/1718487.1718490
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/1718487.1718491
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/1718487.1718492
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/1718487.1718493
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/1718487.1718495
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/1718487.1718496
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/1718487.1718497
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/1718487.1718498
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/1718487.1718499
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/1718487.1718501
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/1718487.1718502
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/1718487.1718503
connecting..

success...
https://dl.acm.org/doi/abs/10.1145/1935826.1935905
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/1935826.1935906
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/1935826.1935907
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/1935826.1935909
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/1935826.1935910
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/1935826.1935911
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/1935826.1935912
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/1935826.1935913
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/1935826.1935914
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/1935826.1935915
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/1935826.1935916
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/1935826.1935917
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/1935826.1935918
connecting...
success...
h

success...
https://dl.acm.org/doi/abs/10.1145/2124295.2124389
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/2124295.2124391
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/2124295.2124392
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/2124295.2124393
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/2124295.2124395
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/2124295.2124396
WSDM 2012 1721.07030916214
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/2433396.2433398
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/2433396.2433400
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/2433396.2433401
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/2433396.2433402
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/2433396.2433403
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/2433396.2433404
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/2433396.2433405

https://dl.acm.org/doi/abs/10.1145/2556195.2556221
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/2556195.2556267
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/2556195.2556256
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/2556195.2556234
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/2556195.2556214
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/2556195.2556242
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/2556195.2556237
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/2556195.2556240
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/2556195.2556258
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/2556195.2556257
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/2556195.2556264
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/2556195.2556235
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/2556195.2556252
connecting...
success...
https://dl.a

success...
https://dl.acm.org/doi/abs/10.1145/2684822.2685302
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/2684822.2685287
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/2684822.2685305
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/2684822.2685322
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/2684822.2685313
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/2684822.2697045
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/2684822.2697048
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/2684822.2685315
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/2684822.2685307
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/2684822.2685310
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/2684822.2685288
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/2684822.2685298
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/2684822.2685312
connecting...
success...
h

success...
https://dl.acm.org/doi/abs/10.1145/2835776.2855116
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/2835776.2855117
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/2835776.2855118
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/2835776.2855080
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/2835776.2855081
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/2835776.2855082
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/2835776.2855083
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/2835776.2855084
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/2835776.2855085
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/2835776.2855086
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/2835776.2855087
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/2835776.2855088
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/2835776.2855089
connecting...
success...
h

success...
https://dl.acm.org/doi/abs/10.1145/3018661.3022761
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/3018661.3022762
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/3018661.3022747
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/3018661.3022748
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/3018661.3022749
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/3018661.3022750
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/3018661.3024933
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/3018661.3022751
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/3018661.3022752
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/3018661.3022753
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/3018661.3022754
WSDM 2017 1743.3350768089294
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/3159652.3160603
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/3159652.31606

success...
https://dl.acm.org/doi/abs/10.1145/3159652.3160587
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/3159652.3160588
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/3159652.3162007
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/3159652.3162006
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/3159652.3162009
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/3159652.3162010
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/3159652.3162008
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/3159652.3162004
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/3159652.3162005
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/3159652.3162011
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/3159652.3160594
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/3159652.3160596
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/3159652.3160591
connecting...
success...
h

success...
https://dl.acm.org/doi/abs/10.1145/3289600.3290614
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/3289600.3290620
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/3289600.3290610
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/3289600.3290615
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/3289600.3290612
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/3289600.3290611
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/3289600.3290609
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/3289600.3290618
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/3289600.3290619
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/3289600.3290617
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/3289600.3290616
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/3289600.3291598
connecting...
success...
https://dl.acm.org/doi/abs/10.1145/3289600.3291603
connecting...
success...
h

In [9]:
with open(r'C:\Users\haiyuanzhao\Desktop\tt.json','a') as f:
    json.dump([1,2,3,4,5],f)

In [ ]:
write_file2(paper_info_list,conf,yr)